In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

examples = [{
        "country": "France",
        "answer": """
        I know this:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    }]



example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about{country}?"),
        ("ai", "{answer}")
    ]
)

examples = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a tourist expert"),
    examples,
    ("human", "What do you know about{country}?")
])



chain = final_prompt | chat

chain.invoke({"country": "Germany"})

C:\Users\dlaru\AppData\Local\Temp\ipykernel_19748\3535656369.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(



        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessage(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-aa694722-dd15-4165-8ed4-953b8e0553d1-0')

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough 

llm = ChatOpenAI(
    temperature= 0.1,
    model_name="gpt-3.5-turbo",
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    memory_key="chat_history",
    return_messages=True
)


examples = [{
    "movie": "Top Gun",
    "answer": "🛩️👨‍✈️🔥"
    },
    {
    "movie": "Godfather",
    "answer": "👨‍👨‍👦🔫🍝"
    }]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Reply with three emojis that represent the movie {movie}."),
    ("ai", "{answer}")

])

examples = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI specialized in expressing a movie with exactly three emojis. Do not add any explanations or text."),
    examples,
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}.")
])

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory)| final_prompt | llm

def invoke_chain(question):
    result=chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result.content)


In [29]:
invoke_chain("Iron man")

🦸‍♂️💥🔧


In [30]:
invoke_chain("Starwars")

⚔️🌌🤖


In [32]:
invoke_chain("what was the first movie I asked?")


You asked about Iron Man.
